Model typu filtrowanie kolaboracyjne (collaborative filtering).

Wykorzystujemy w tym celu macierz interakcji, którą następnie dekomponujemy na macierze mniejszej wymiarowości z użyciem 
TruncatedSVD - sklearn. 

In [72]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

Przygotowanie macierzy interakcji

In [73]:
sessionsDataPath = '../notebooks/data/v2/sessions.jsonl'
productsDataPath = '../notebooks/data/v2/products.jsonl'
sessionsDF = pd.read_json(sessionsDataPath, lines=True)
productsDF = pd.read_json(productsDataPath, lines=True)

df = sessionsDF.drop(columns=["session_id", "timestamp", "event_type", "offered_discount", "purchase_id"])
df["count"] = 1

#interaction matrix is build on part of the sessionDF 
train, test = train_test_split(df, test_size=0.2, stratify=df['product_id'])

interactionMatrixDF = pd.pivot_table(train, index="user_id", columns="product_id", values="count", aggfunc=np.sum, fill_value=0)
display(interactionMatrixDF)

product_id,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,1310,1311,1312,1313,1314,1315,1316,1317,1318,1319
user_id,,,,,,,,,,,,,,,,,,,,,
102,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103,0,2,1,15,1,2,1,4,1,8,...,0,1,1,1,1,3,4,1,0,5
104,0,1,6,3,5,9,0,3,3,4,...,1,2,0,0,0,0,7,2,6,1
105,0,1,6,6,2,4,0,2,2,3,...,0,0,1,0,2,1,9,5,0,1
106,2,1,0,8,2,2,1,1,1,4,...,0,0,2,3,1,1,4,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2,0,6,11,8,2,0,2,6,5,...,0,2,0,2,0,1,9,1,3,3
298,0,0,5,1,1,1,0,3,1,1,...,1,0,0,0,1,1,8,3,0,0
299,0,0,3,0,0,0,0,1,0,0,...,0,0,1,0,2,0,3,1,0,0


Normalizacja wartości w macierzy interakcji.

In [74]:
interactionMatrixDF = pd.DataFrame(preprocessing.MinMaxScaler().fit_transform(interactionMatrixDF))
display(interactionMatrixDF)

,0,1,2,3,4,5,6,7,8,9,...,309,310,311,312,313,314,315,316,317,318
0,0.000000,0.0,0.0000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.0000,...,0.000000,0.000000,0.0,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.0
1,0.000000,0.4,0.0625,0.625000,0.090909,0.181818,0.25,0.363636,0.1,0.5000,...,0.000000,0.142857,0.2,0.25,0.142857,0.75,0.16,0.142857,0.000000,1.0
2,0.000000,0.2,0.3750,0.125000,0.454545,0.818182,0.00,0.272727,0.3,0.2500,...,0.333333,0.285714,0.0,0.00,0.000000,0.00,0.28,0.285714,0.666667,0.2
3,0.000000,0.2,0.3750,0.250000,0.181818,0.363636,0.00,0.181818,0.2,0.1875,...,0.000000,0.000000,0.2,0.00,0.285714,0.25,0.36,0.714286,0.000000,0.2
4,0.285714,0.2,0.0000,0.333333,0.181818,0.181818,0.25,0.090909,0.1,0.2500,...,0.000000,0.000000,0.4,0.75,0.142857,0.25,0.16,0.428571,0.000000,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0.285714,0.0,0.3750,0.458333,0.727273,0.181818,0.00,0.181818,0.6,0.3125,...,0.000000,0.285714,0.0,0.50,0.000000,0.25,0.36,0.142857,0.333333,0.6
196,0.000000,0.0,0.3125,0.041667,0.090909,0.090909,0.00,0.272727,0.1,0.0625,...,0.333333,0.000000,0.0,0.00,0.142857,0.25,0.32,0.428571,0.000000,0.0
197,0.000000,0.0,0.1875,0.000000,0.000000,0.000000,0.00,0.090909,0.0,0.0000,...,0.000000,0.000000,0.2,0.00,0.285714,0.00,0.12,0.142857,0.000000,0.0
198,0.285714,0.6,0.5000,0.416667,0.272727,0.181818,0.00,0.454545,0.4,0.3750,...,0.666667,0.142857,0.0,0.00,0.428571,0.50,0.28,0.142857,0.000000,0.6


Utworzenie słownika dla szybszego wyszukiwania.

In [75]:
idxNameDict = pd.Series(productsDF["product_id"].values, index=productsDF.index).to_dict()

Przygotowanie zbirów danych: treningowy i testowy

In [76]:
from sklearn.model_selection import train_test_split

trainUser, testUser = train_test_split(interactionMatrixDF, test_size=0.2)

#transpose because items are columns
# trainItem, testItem = train_test_split(interactionMatrixDF.transpose(), test_size=0.2)
# trainItem = interactionMatrixDF.transpose() #for testing only




Dekompozycja utworzonej macierzy na podmacierze ze względu na użytkowników i produkty.

In [77]:
from sklearn.decomposition import TruncatedSVD

#initial hiperparameters
epsilon = 1e-9
latentFactors = 10

#generate item latent features
itemSVD = TruncatedSVD(n_components=latentFactors)
itemFeatures = itemSVD.fit_transform(interactionMatrixDF.transpose()) + epsilon #transpose because items are columns

#generate user latent features
userSVD = TruncatedSVD(n_components=latentFactors)
userFeatures = userSVD.fit_transform(interactionMatrixDF) + epsilon

pd.DataFrame(userFeatures)
pd.DataFrame(itemFeatures)


,0,1,2,3,4,5,6,7,8,9
0,1.664556,0.172227,0.364377,-0.033091,-0.695895,-0.452172,-0.234337,0.284573,0.194180,0.125344
1,2.752263,0.020944,-0.423481,-0.039406,0.255529,0.198875,-0.390085,-0.104170,-0.391972,0.312133
2,3.370161,-0.423574,-0.129507,-0.093683,0.404711,-0.084911,-0.020320,-0.239620,-0.062518,0.317313
3,4.323772,-0.337577,0.044173,-0.329524,-0.054429,-0.039535,-0.065743,-0.406101,-0.507461,-0.456936
4,4.668519,0.818677,-0.051278,-0.802611,0.387646,-0.330193,-0.549958,-0.097009,-0.599519,0.327598
...,...,...,...,...,...,...,...,...,...,...
314,2.851270,0.573231,-0.023809,0.733136,0.201436,-0.158158,-0.806149,0.380769,-0.090424,-0.005838
315,3.611771,0.239298,-0.103178,-0.034476,0.277319,-0.277200,-0.326032,0.100991,-0.064231,-0.043644
316,2.936040,-0.347837,0.117314,-0.169859,0.164449,-0.031517,-0.087226,0.008759,0.325555,0.459430
317,1.509518,0.171151,-0.365250,-0.394312,-0.333616,0.149139,0.167650,-0.092469,0.284779,0.097168


In [78]:
def top_k(item_id, top_k, corr_mat, map_name):
    topItems = corr_mat[item_id,:].argsort()[-top_k:][::-1]
    topItems = [map_name[e] for e in topItems]
    return topItems

In [79]:
from sklearn.metrics.pairwise import cosine_similarity

itemCorrMat = cosine_similarity(itemFeatures)

#before test_split it is mandatory to create dict mapping indexes of products in productsDF to labels or ids
#because truncatedSVD has rows coresponding to rows in productsDF, but after split there not the same
recommendations = top_k(5, 10, itemCorrMat, idxNameDict)
display(productsDF.loc[productsDF['product_id'].isin(recommendations)])

,product_id,product_name,category_path,price,user_rating
2,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7639.00,1.493143
3,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99,4.063970
5,1006,Call of Duty 4 Modern Warfare (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,59.90,3.054270
7,1008,Tom Clancy&#39;s Rainbow Six Vegas (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99,3.715598
9,1010,BioShock 2 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,89.99,3.510874
10,1011,BioShock Infinite (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,139.99,3.251818
11,1012,Fallout New Vegas (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,69.00,2.386605
12,1013,LA Noire (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,129.99,0.166660
298,1299,Vivanco TVA 301,Sprzęt RTV;Video;Telewizory i akcesoria;Anteny...,109.00,1.874503
313,1314,Assassin&#39;s Creed (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99,2.416163
